In [2]:
!pip install pandas

     ---------------------------------------- 10.6/10.6 MB 9.2 MB/s eta 0:00:00


In [4]:
!pip install nltk

     ---------------------------------------- 1.5/1.5 MB 5.6 MB/s eta 0:00:00
     -------------------------------------- 307.0/307.0 KB 6.3 MB/s eta 0:00:00
     ------------------------------------- 262.0/262.0 KB 16.8 MB/s eta 0:00:00
     ---------------------------------------- 96.6/96.6 KB 5.4 MB/s eta 0:00:00
     ---------------------------------------- 78.4/78.4 KB 4.5 MB/s eta 0:00:00


In [27]:
!pip install sklearn

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 7.2/7.2 MB 4.7 MB/s eta 0:00:00
     ---------------------------------------- 37.0/37.0 MB 4.5 MB/s eta 0:00:00
Using legacy 'setup.py install' for sklearn, since package 'wheel' is not installed.
  Running setup.py install for sklearn: started
  Running setup.py install for sklearn: finished with status 'done'


In [38]:
nltk.download('stopwords') # to use stopwords
nltk.download('punkt') # to use word_tokenize

import pandas as pd
import numpy as np

import nltk

import re #регулярные выражения
import math
from collections import Counter
import requests
import time
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя'

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\wequalwo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\wequalwo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [52]:
# служебная функция для расчета занимаемой памяти 
BYTES_TO_MB_DIV = 0.000001
def print_memory_usage_of_data_frame(df):
    mem = round(df.memory_usage().sum() * BYTES_TO_MB_DIV, 3) 
    print("Memory usage is " + str(mem) + " MB")

In [41]:
# десмонтсранция идеи токенизации и стоп-слов
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords as stopwords
from nltk.tokenize import word_tokenize

example_sent = """Это простое предложение показывает фильтрацию на стоп-слова"""
 
stop_words = set(stopwords.words('russian'))
word_tokens = word_tokenize(example_sent)
filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
filtered_sentence = []
 
for w in word_tokens:
    if w not in stop_words:
        filtered_sentence.append(w)
 
print(word_tokens)
print(filtered_sentence)

from nltk.corpus import stopwords
print("Стоп-слова русского языка:")
print(stopwords.words('russian'))

['Это', 'простое', 'предложение', 'показывает', 'фильтрацию', 'на', 'стоп-слова']
['Это', 'простое', 'предложение', 'показывает', 'фильтрацию', 'стоп-слова']
Стоп-слова русского языка:
['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'мож

## Создадим функции, позволяющие генерировать следующие ошибки в словах:

1.   **Пропуск буквы:** ${резонанс - рзонанс}$
2.   **Дублирование буквы:** ${резонанс - реезонанс}$
3.   **Перестановка букв:** ${резонанс - рзеонанс}$
4.   **Опечатка:** ${резонанс - ркзонанс}$
5.   **Некоторые орфографические ошибки:**  ${резонанс - ризонанс}$


In [42]:
# все ошибки на расстоянии 2 (расстояние Левинштейна)
def distance2(word):
  return {e2 for e1 in distance1(word) if e1 for e2 in distance1(e1)}

# все ошибки на расстоянии 1 (расстояние Левинштейна)
def distance1(word):
    pairs      = splits(word)
    transposes = [a+b[1]+b[0]+b[2:]  for (a,b) in pairs if len(b)>1]                           # перестановки
    replaces   = [a+c+b[1:]          for (a,b) in pairs if b for c in replaces_set[b[0]] if b] # замены: 
                                                                                                        # опечатки, 
                                                                                                        # пропуски и дублирования букв, 
                                                                                                        # орфографические ошибки

    last_replaces = [word[0:-1] + c for c in replaces_set[word[-1]]]                           # замены в конце слова
    return set(transposes + replaces + last_replaces)

def splits(word):
    return [(word[:i], word[i:])
            for i in range (len(word)+1)]

# список возможных замен в слове
replaces_set = pd.Series([          
               ['с', 'в', 'у', 'к', 'е', 'п', 'м', 'о', 'аа', ''],
               ['ь', 'о', 'л', 'д', 'ж', 'ю', 'п', 'бб', ''],
               ['ч', 'ы', 'ц', 'у', 'к', 'а', 'с', 'вв', 'ф', ''],
               ['р', 'н', 'ш', 'л', 'о', 'гг', 'к', ''],
               ['б', 'л', 'ш', 'щ', 'з', 'ж', 'ю', 'дд', 'т', ''],
               ['а', 'к', 'н', 'р', 'п', 'ее', 'и', 'ё', 'о', ''],
               ['ёё', 'й', 'йо', 'е', ''],
               ['ю', 'д', 'щ', 'з', 'х', 'э', 'жж', 'ш', ''],
               ['д', 'щ', 'х', 'э', 'ж', 'зз', 'с', ''],
               ['м', 'а', 'п', 'р', 'т', 'ии', 'е', 'ы', ''],
               ['ц', 'ы', 'ф', 'йй',''],
               ['а', 'в', 'у', 'е', 'п', 'п', 'а', 'кк', 'г', ''],
               ['ь', 'о', 'г', 'ш', 'щ', 'д', 'б', 'лл', ''],
               ['с', 'а', 'п', 'и', 'мм', 'н', ''],
               ['р', 'п', 'е', 'г', 'о', 'нн', 'м', ''],
               ['т', 'р', 'н', 'г', 'ш', 'л', 'ь', 'оо', 'а', 'е', 'ё', 'у', ''],
               ['м', 'а', 'к', 'е', 'н', 'р', 'и', 'пп', 'б', ''],
               ['и', 'п', 'е', 'н', 'г', 'о', 'т', 'рр', 'р', ''],
               ['ч', 'в', 'а', 'м', 'сс', 'з', ''],
               ['и', 'п', 'р', 'о', 'ь', 'тт', 'д', ''],
               ['в', 'ы', 'ц', 'к', 'а', 'уу', 'ю', 'о', ''],
               ['я', 'ч', 'ы', 'ц', 'й', 'фф', 'в', ''],
               ['ж', 'з', 'ъ', 'э', 'хх', ''],
               ['ы', 'ф', 'й', 'у', 'в', 'ы', 'цц', 'ч', ''],
               ['я', 'ф', 'ы', 'в', 'с', 'чч', ''],
               ['л', 'о', 'г', 'щ', 'д', 'шш', 'щ', 'шь', 'щь', 'ж', ''],
               ['л', 'ш', 'з', 'ж', 'д', 'щщ', 'щь', 'шь', 'ж', ''],
               ['э', 'х', 'ъъ', 'ь', ''],
               ['ч', 'я', 'ф', 'й', 'ц', 'у', 'в', 'и', 'ыы', ''],
               ['т', 'о', 'л', 'б', 'ьь', 'ъ', ''],
               ['ж', 'з', 'х', 'ъ', ''],
               ['б', 'л', 'д', 'ж', 'э', 'у', 'йу', ''],
               ['ф', 'ы', 'ч', 'яя', 'а', 'йа', ''],
               ], 
               index = ['а','б','в','г', 'д', 'е', 'ё', 'ж', 'з', 'и', 'й', 'к', 'л', 'м', 'н', 'о', 'п', 'р', 'с', 'т', 'у', 'ф', 'х', 'ц', 'ч', 'ш', 'щ', 'ъ', 'ы', 'ь', 'э', 'ю', 'я'])


**Импортируем текст**

In [45]:
import codecs
with codecs.open('top10000.txt', 'r', encoding='utf-8') as file:
    TEXT = file.read().replace('\n', ' ')

def tokens(text):
    return re.findall(r'[а-я]+', text.lower())
list_of_words = tokens(TEXT)

# удалим все слова, короче 3 символов (потому что слова из 2х символов неинформативны):
tmp = [w for w in list_of_words if len(w) > 2]
list_of_words = tmp

# пока ограничимся 500 словами
del list_of_words[500:]

# выведем первые 20 слов
print(list_of_words[:20])

['что', 'тот', 'быть', 'весь', 'это', 'как', 'она', 'они', 'так', 'сказать', 'этот', 'который', 'может', 'человек', 'один', 'еще', 'такой', 'только', 'себя', 'свое']


**Для каждого слова рассчитаем возможыне ошибки на расстоянии 1 или 2. Затем произведем токенизацию**

In [46]:
words = []
for item in list_of_words:
  words.append([item, distance1(item).union(distance2(item))])# ??? как оптимизировать - неясно

In [9]:
def _tokenize(word):
  return " ".join([a + a for a in word])
_tokenize('привет')

'пп рр ии вв ее тт'

In [10]:
tokenized = []
to_vec = []
for w in words:
  tokens = []
  for errors in w[1]:
    st = _tokenize(errors)
    tokens.append(st)
    to_vec.append(st)
  tokenized.append([w[0], tokens])

**Вектризация текста посредством *TfidfVectorizer (TF-IDF)***

In [89]:
# print(to_vec[400:450])
from sklearn.feature_extraction.text import TfidfVectorizer
corpus = to_vec
vectorizer = TfidfVectorizer()
V = vectorizer.fit_transform(corpus)
print("Найденные токены:")
print(vectorizer.get_feature_names_out())

Найденные токены:
['аа' 'бб' 'вв' 'гг' 'дд' 'ее' 'жж' 'зз' 'ии' 'йй' 'кк' 'лл' 'мм' 'нн'
 'оо' 'пп' 'рр' 'сс' 'тт' 'уу' 'фф' 'хх' 'цц' 'чч' 'шш' 'щщ' 'ъъ' 'ыы'
 'ьь' 'ээ' 'юю' 'яя' 'ёё']


In [49]:
# для наглядности создадим dataframe с данными
df = pd.DataFrame(V.toarray())

***bool* - векторизация:**

In [92]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_bin = CountVectorizer(binary = True)
V_bin = vectorizer_bin.fit_transform(to_vec)
print("Найденные токены:")
print(vectorizer_bin.get_feature_names_out())
#print(V_bin.toarray())

Найденные токены:
['аа' 'бб' 'вв' 'гг' 'дд' 'ее' 'жж' 'зз' 'ии' 'йй' 'кк' 'лл' 'мм' 'нн'
 'оо' 'пп' 'рр' 'сс' 'тт' 'уу' 'фф' 'хх' 'цц' 'чч' 'шш' 'щщ' 'ъъ' 'ыы'
 'ьь' 'ээ' 'юю' 'яя' 'ёё']


**Определяем метки для каждого ошибочного слова**

In [57]:
labels = []
_words = []
for i in range(0, len(tokenized)):
  for j in range(0, len(tokenized[i][1])):
    labels.append(i)
    _words.append(tokenized[i][0])
df["label"] = labels

In [58]:
df

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,31,32,label
0,0.000000,0.0,0.000000,0.623857,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0
1,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.474597,0.0,0.0,0.0,0.0,0
2,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.691239,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0
3,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0
4,0.436127,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
884053,0.244087,0.0,0.299210,0.000000,0.0,0.0,0.0,0.0,0.273316,0.000000,...,0.000000,0.0,0.0,0.767646,0.000000,0.0,0.0,0.0,0.0,499
884054,0.243597,0.0,0.298610,0.000000,0.0,0.0,0.0,0.0,0.272768,0.341725,...,0.000000,0.0,0.0,0.383053,0.000000,0.0,0.0,0.0,0.0,499
884055,0.294356,0.0,0.360831,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.000000,0.0,0.0,0.462871,0.000000,0.0,0.0,0.0,0.0,499
884056,0.224460,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.251339,0.314879,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,499


In [59]:
from sklearn.model_selection import train_test_split
from sklearn.metrics  import f1_score, accuracy_score


from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression

**Попробуем несколько сжать данные**

In [60]:
def convert_to_sparse_pandas(df, exclude_columns = []):
    df = df.copy()
    exclude_columns = set(exclude_columns)

    for (columnName, columnData) in df.iteritems():
        if columnName in exclude_columns:
            continue
        df[columnName] = pd.arrays.SparseArray(columnData.values, dtype = np.float16)

    return df

df_sparse = convert_to_sparse_pandas(df, exclude_columns = ["label"])
# display(df_sparse.dtypes)
print_memory_usage_of_data_frame(df_sparse)

Memory usage is 182.116 MB


In [93]:
y = df['label']
X = df[df.columns.difference(['label'])]

y_sparse = df_sparse['label']
X_sparse = df_sparse[df_sparse.columns.difference(['label'])]

y_csr = df['label']
X_csr = V

y_binary = df['label']
X_binary = V_bin

print('X:')
print_memory_usage_of_data_frame(X)
print('X_sparse:')
print_memory_usage_of_data_frame(X_sparse)
print('X_csr:')
get_csr_memory_usage(X_csr)
print('X_binary:')
get_csr_memory_usage(X_binary)

X:
Memory usage is 233.391 MB
X_sparse:
Memory usage is 175.044 MB
X_csr:
Memory usage is 68.581552 MB
X_binary:
Memory usage is 68.581552 MB


**Как видим, самый оптимальный вариант - разреженная матрица**

In [64]:
'''from scipy.sparse import lil_matrix

def data_frame_to_scipy_sparse_matrix(df):
    arr = lil_matrix(df.shape, dtype = np.float32)
    for i, col in enumerate(df.columns):
        ix = df[col] != 0
        arr[np.where(ix), i] = 1
    return arr.tocsr()

def get_csr_memory_usage(matrix):
    mem = (X_csr.data.nbytes + X_csr.indptr.nbytes + X_csr.indices.nbytes) * BYTES_TO_MB_DIV
    print("Memory usage is " + str(mem) + " MB")'''

## Перейдем к обучению модели

In [131]:
models = []
list_of_names = []
X_tests_arr = []
Y_tests_arr = []

In [132]:
model = DecisionTreeClassifier()
# 'Sparse pandas dataframe': [X_sparse, y_sparse],
vector_dict = {'Pandas dataframe': [X, y],
    'Scipy sparse matrix': [X_csr, y_csr],
    'Scipy sparse matrix binary': [X_binary, y_binary]
    }

for key, item in vector_dict.items():
    print(key)
    list_of_names.append(key)
    start = time.time()
    if (key != 'Pandas dataframe'):
        XX = item[0].toarray()
    else:
        XX = X
        
    X_train, X_test, y_train, y_test = train_test_split(XX, y, test_size = 0.8, random_state = 42)
    end = time.time()
    duration = round(end - start, 2)
    print("Train-test split: " + str(duration) + " secs")

    start = time.time()
    model.fit(X_train, y_train)

    end = time.time()
    duration = round(end - start, 2)
    
    models.append(model)
    X_tests_arr.append(X_test)
    Y_tests_arr.append(y_test)
    print("Training: " + str(duration) + " secs")
    print("\n")

Pandas dataframe
Train-test split: 0.22 secs
Training: 16.03 secs


Scipy sparse matrix
Train-test split: 0.22 secs
Training: 17.48 secs


Scipy sparse matrix binary
Train-test split: 0.22 secs
Training: 1.9 secs




**Теперь протестируем обученную модель**

In [133]:
def test(mod, Xtest, Ytest):
    start = time.time()
    pred = mod.predict(Xtest)
    end = time.time()
    duration = round(end - start, 2)
    print("Testing: " + str(duration) + " secs")
    print('accuracy:', accuracy_score(Ytest, pred))
    print('      f1:', f1_score(Ytest, pred, average = 'macro'))
    print('    mean:', mean_absolute_error(Ytest, pred))
    print("\n")

In [134]:
for i in range(len(models)):
    print(list_of_names[i])
    test(models[i], X_tests_arr[i], Y_tests_arr[i])

Pandas dataframe
Testing: 1.63 secs
accuracy: 0.009674130819925712
      f1: 0.013150407862761483
    mean: 227.39396985777248


Scipy sparse matrix
Testing: 1.43 secs
accuracy: 0.009674130819925712
      f1: 0.013150407862761483
    mean: 227.39396985777248


Scipy sparse matrix binary
Testing: 1.68 secs
accuracy: 0.6571155480334311
      f1: 0.5426343007573801
    mean: 53.44590503741974




## Проведем "живой" тест на бинароной модели

In [ ]:
pin = input()
create_vec = 

## bin

In [ ]:
start = time.time()
pred = model.predict(X_test)
end = time.time()
duration = round(end - start, 2)
print("Testing: " + str(duration) + " secs")
print("\n")

In [105]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

print(accuracy_score(y_test, pred))
print(f1_score(y_test, pred, average = 'macro'))

print(mean_absolute_error(y_test, pred))

0.6571622078283824
0.5426987035755111
53.354150671547565
